In [8]:
import pandas as pd
import requests
from io import StringIO

In [9]:
url = "https://db.netkeiba.com/race/201901010101"
pd.read_html(url, encoding='euc-jp')[0]

/Users/nasutougo/Desktop/racing_tip/practice/project_env/lib/python3.10/site-packages/bs4/__init__.py:228: UserWarning: You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.
  warnings.warn("You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.")


,着 順,枠 番,馬 番,馬名,性齢,斤量,騎手,タイム,着差,ﾀｲﾑ 指数,...,上り,単勝,人 気,馬体重,調教 ﾀｲﾑ,厩舎 ｺﾒﾝﾄ,備考,調教師,馬主,賞金 (万円)
0,1,1,1,ゴルコンダ,牡2,54,ルメール,1:48.3,NaN,**,...,36.5,1.4,1,518(-16),NaN,NaN,NaN,[東] 木村哲也,サンデーレーシング,500.0
1,2,3,3,プントファイヤー,牡2,54,岩田康誠,1:50.1,大,**,...,38.2,3.5,2,496(-8),NaN,NaN,NaN,[東] 手塚貴久,嶋田賢,200.0
2,3,4,4,ラグリマスネグラス,牡2,51,団野大成,1:50.9,5,**,...,38.8,46.6,6,546(+6),NaN,NaN,NaN,[東] 藤沢和雄,多田信尊,130.0
3,4,8,9,キタノコドウ,牡2,51,菅原明良,1:51.5,3.1/2,**,...,38.5,56.8,7,458(-8),NaN,NaN,NaN,[東] 高木登,北所直人,75.0
4,5,5,5,ネモフィラブルー,牡2,54,川島信二,1:51.7,1.1/2,**,...,38.9,140.3,9,436(0),NaN,NaN,NaN,[西] 矢作芳人,青芝商事,50.0
5,6,8,8,マイネルラクスマン,牡2,54,丹内祐次,1:52.1,2.1/2,**,...,40.1,9.7,3,480(+8),NaN,NaN,NaN,[東] 金成貴史,サラブレッドクラブ・ラフィアン,NaN
6,7,2,2,サンモンテベロ,牝2,54,黛弘人,1:52.5,2.1/2,**,...,39.6,114.7,8,450(+2),NaN,NaN,NaN,[東] 中野栄治,加藤ステーブル,NaN
7,8,7,7,エスカレーション,牝2,54,藤岡佑介,1:52.5,アタマ,**,...,39.5,26.1,5,448(-4),NaN,NaN,NaN,[東] 高柳瑞樹,松田整二,NaN
8,9,6,6,セイウンジュリア,牝2,54,池添謙一,1:52.6,クビ,**,...,40.3,16.4,4,470(0),NaN,NaN,NaN,[西] 浅見秀一,西山茂行,NaN


01:札幌, 02:函館, 03:福島, 04:新潟, 05:東京, 06:中山, 07:中京, 08:京都, 09:阪神, 10:小倉

In [10]:
import time
from tqdm import tqdm

def scrape_race_results(race_id_list, pre_race_results = {}):
  # 辞書としてrace_resultsを初期化
  # race_results = {}
  race_results = pre_race_results
  for race_id in tqdm(race_id_list):
    if race_id in race_results.keys(): # 
      continue
    try:
      url = 'https://db.netkeiba.com/race/' + race_id
      response = requests.get(url)

      # response.contentから得られるバイナリデータを'euc-jp'でデコード
      content = response.content.decode('euc-jp')

      #StringIOを使用してデコードされたHTMLコンテンツをpandasに渡す
      df = pd.read_html(StringIO(content))[0]
      
      # race_idをキーとして結果を辞書に格納
      race_results[race_id] = df
      time.sleep(1)
    except IndexError: # テーブルがない場合のエラー処理
      continue
    except:
      break


  return race_results

In [11]:
# レースIDのリスト
race_id_list = ["201902010101", "201902010102", "201902010103"]

# 関数をテスト
test = scrape_race_results(race_id_list)

100%|██████████| 3/3 [00:08<00:00,  2.77s/it]


In [12]:
race_id_list = []
for place in range(1,11,1):
  for kai in range(1,6,1):
    for day in range(1,9,1):
      for r in range(1,13,1):
        race_id = "2019" + str(place).zfill(2) + str(kai).zfill(2) + str(day).zfill(2) + str(r).zfill(2)
        race_id_list.append(race_id)

In [13]:
len(race_id_list)

4800

In [22]:
test2 = scrape_race_results(race_id_list, test1)

100%|██████████| 4800/4800 [1:42:57<00:00,  1.29s/it]  


In [23]:
for key in test2.keys():
  test2[key].index = [key] * len(test2[key])

In [25]:
results = pd.concat([test2[key] for key in test2.keys()], sort=False)

In [26]:
results.to_pickle('results.pickle')

In [89]:
results = pd.read_pickle('results.pickle')

In [31]:
results

,着 順,枠 番,馬 番,馬名,性齢,斤量,騎手,タイム,着差,単勝,人 気,馬体重,調教師
201902010101,1,6,12,アドマイヤデルタ,牡3,56.0,吉田隼人,1:08.9,NaN,4.0,3.0,490(+32),[西] 西村真幸
201902010101,2,3,5,ティレニア,牝3,54.0,藤岡佑介,1:09.1,1.1/2,3.7,2.0,442(+4),[西] 高橋亮
201902010101,3,4,7,ドゥシャンパーニュ,牝3,54.0,北村友一,1:09.3,1,2.9,1.0,428(-4),[西] 安田翔伍
201902010101,4,5,10,シャイニーブランコ,牡3,56.0,岩田康誠,1:09.5,1.1/2,10.3,5.0,442(+2),[西] 石坂正
201902010101,5,4,8,シュンカジョウ,牝3,54.0,松岡正海,1:09.6,クビ,57.5,8.0,438(+14),[西] 藤岡健一
...,...,...,...,...,...,...,...,...,...,...,...,...,...
201910020812,10,7,12,オーシャンスケイプ,牝3,52.0,大野拓弥,1:49.8,3/4,45.7,10.0,414(+6),[西] 西村真幸
201910020812,11,3,3,プリヴェット,牝4,55.0,藤岡康太,1:50.3,3,29.8,9.0,486(+2),[西] 今野貞一
201910020812,12,5,7,アドマイヤクィーン,牝4,55.0,松山弘平,1:50.3,クビ,29.6,8.0,476(+6),[西] 須貝尚介
201910020812,13,8,13,タムロドリーム,牝3,49.0,亀田温心,1:50.4,1/2,134.8,12.0,472(+2),[西] 西園正都


## データ整形、前処理

In [34]:
print(results.columns)

Index(['着 順', '枠 番', '馬 番', '馬名', '性齢', '斤量', '騎手', 'タイム', '着差', '単勝', '人 気',
       '馬体重', '調教師'],
      dtype='object')


In [35]:
results['着 順'].value_counts()

着 順
4       2864
1       2863
3       2859
2       2857
5       2855
6       2853
7       2836
8       2801
9       2702
10      2553
11      2372
12      2161
13      1938
14      1702
15      1403
16      1101
1        330
4        330
3        329
6        329
2        328
5        328
7        327
8        322
9        307
10       283
11       263
12       229
中        209
17       198
13       197
18       165
除        157
14       157
15       120
取         57
16        26
17        22
2(降)       2
5(降)       1
18         1
3(降)       1
Name: count, dtype: int64

In [36]:
results['着 順'].astype(str).value_counts()

着 順
4       3194
1       3193
3       3188
2       3185
5       3183
6       3182
7       3163
8       3123
9       3009
10      2836
11      2635
12      2390
13      2135
14      1859
15      1523
16      1127
17       220
中        209
18       166
除        157
取         57
2(降)       2
5(降)       1
3(降)       1
Name: count, dtype: int64

In [43]:
results2 = results[~(results['着 順'].astype(str).str.contains('\D'))] # ~は否定を示す,着順で1~9以外を省いたデータを取得

In [47]:
results2.columns

Index(['着 順', '枠 番', '馬 番', '馬名', '性齢', '斤量', '騎手', 'タイム', '着差', '単勝', '人 気',
       '馬体重', '調教師'],
      dtype='object')

In [52]:
results2['性齢'].value_counts() #value_counts()で最多順番で並び替え

性齢
牡3     9169
牝3     8394
牡2     4333
牡4     3969
牝2     3310
牡5     3164
牝4     3105
牝5     1952
牡6     1719
牡7      869
牝6      681
セ4      433
セ5      426
セ6      398
セ3      387
牡8      353
セ7      189
牝7      140
牡9       80
セ8       76
セ2       61
牝8       57
牡10      20
牝10       7
セ9        7
セ10       5
セ11       3
牡11       2
セ12       2
Name: count, dtype: int64

In [54]:
results2['性'] = results2['性齢'].map(lambda x: str(x)[0]) # xに値を代入していく

/var/folders/gb/btg706891vzdbwsvzgv2bt5h0000gn/T/ipykernel_15212/3405577624.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results2['性'] = results2['性齢'].map(lambda x: str(x)[0]) # xに値を代入していく


In [57]:
results2['年齢'] = results2['性齢'].map(lambda x: str(x)[1:])

/var/folders/gb/btg706891vzdbwsvzgv2bt5h0000gn/T/ipykernel_15212/2218314994.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results2['年齢'] = results2['性齢'].map(lambda x: str(x)[1:])


In [62]:
results2['体重'] = results2['馬体重'].str.split('(', expand = True )[0].astype(int) #数字型に変換

/var/folders/gb/btg706891vzdbwsvzgv2bt5h0000gn/T/ipykernel_15212/4220505339.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results2['体重'] = results2['馬体重'].str.split('(', expand = True )[0].astype(int) #数字型に変換


In [64]:
results2['体重変化'] = results2['馬体重'].str.split('(', expand = True )[1].str[:-1].astype(int) #数字型に変換

/var/folders/gb/btg706891vzdbwsvzgv2bt5h0000gn/T/ipykernel_15212/923385624.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results2['体重変化'] = results2['馬体重'].str.split('(', expand = True )[1].str[:-1].astype(int) #数字型に変換


In [66]:
results2.drop(['タイム', '着差', '調教師'], axis=1, inplace=True)

/var/folders/gb/btg706891vzdbwsvzgv2bt5h0000gn/T/ipykernel_15212/581360874.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results2.drop(['タイム', '着差', '調教師'], axis=1, inplace=True)


In [73]:
results2['着 順'] = results2['着 順'].astype(int)

/var/folders/gb/btg706891vzdbwsvzgv2bt5h0000gn/T/ipykernel_15212/636693927.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results2['着 順'] = results2['着 順'].astype(int)


In [70]:
results2['単勝'] = results2['単勝'].astype(float)

/var/folders/gb/btg706891vzdbwsvzgv2bt5h0000gn/T/ipykernel_15212/3015014177.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results2['単勝'] = results2['単勝'].astype(float)


In [77]:
results2.drop(['性齢', '馬体重'], axis=1, inplace=True) # 性齢,馬体重をデータから削除

KeyError: "['性齢', '馬体重'] not found in axis"

In [78]:
results2.drop(['着順'], axis=1, inplace=True) # 着順をデータから削除


/var/folders/gb/btg706891vzdbwsvzgv2bt5h0000gn/T/ipykernel_15212/1587478764.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results2.drop(['着順'], axis=1, inplace=True) # 着順をデータから削除


In [81]:
results2['年齢'] = results2['年齢'].astype(int)

/var/folders/gb/btg706891vzdbwsvzgv2bt5h0000gn/T/ipykernel_15212/3551137494.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results2['年齢'] = results2['年齢'].astype(int)


In [95]:
def preprocessing(results):
  df = results.copy() # 関数にいれたデータを保持したまま新しいデータをコピー

  # 着順に数字以外の文字列が含まれているものを取り除く
  df = df[~(df['着 順'].astype(str).str.contains('\D'))] # ~は否定を示す,着順で1~9以外を省いたデータを取得
  df['着 順'] = df['着 順'].astype(int) # 整数型に変換
  
  # 性齢を性と年齢に分ける
  df['性'] = df['性齢'].map(lambda x: str(x)[0]) # xに値を代入していく
  df['年齢'] = df['性齢'].map(lambda x: str(x)[1:]).astype(int) 

  # 馬体重を体重と体重変化に分ける
  df['体重'] = df['馬体重'].str.split('(', expand = True )[0].astype(int) #数字型に変換
  df['体重変化'] = df['馬体重'].str.split('(', expand = True )[1].str[:-1].astype(int) #数字型に変換

  # データをint,floatに変換
  df['単勝'] = df['単勝'].astype(float)

  # 不要な列を削除
  df.drop(['タイム', '着差', '調教師', '年齢', '馬体重'], axis=1, inplace=True)
  
  return df


In [96]:
test = preprocessing(results)

In [97]:
test

,着 順,枠 番,馬 番,馬名,性齢,斤量,騎手,単勝,人 気,性,体重,体重変化
201902010101,1,6,12,アドマイヤデルタ,牡3,56.0,吉田隼人,4.0,3.0,牡,490,32
201902010101,2,3,5,ティレニア,牝3,54.0,藤岡佑介,3.7,2.0,牝,442,4
201902010101,3,4,7,ドゥシャンパーニュ,牝3,54.0,北村友一,2.9,1.0,牝,428,-4
201902010101,4,5,10,シャイニーブランコ,牡3,56.0,岩田康誠,10.3,5.0,牡,442,2
201902010101,5,4,8,シュンカジョウ,牝3,54.0,松岡正海,57.5,8.0,牝,438,14
...,...,...,...,...,...,...,...,...,...,...,...,...
201910020812,10,7,12,オーシャンスケイプ,牝3,52.0,大野拓弥,45.7,10.0,牝,414,6
201910020812,11,3,3,プリヴェット,牝4,55.0,藤岡康太,29.8,9.0,牝,486,2
201910020812,12,5,7,アドマイヤクィーン,牝4,55.0,松山弘平,29.6,8.0,牝,476,6
201910020812,13,8,13,タムロドリーム,牝3,49.0,亀田温心,134.8,12.0,牝,472,2
